In [34]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
sys.path.append('./scripts')  
import preprocesamiento
import feature_engineering
import model_lgb
importlib.reload(preprocesamiento)
importlib.reload(model_lgb)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

##### Experimento 03
Más FE, 50 trials, valores negativos de prediccion se puso el promedio 12 meses

##### Levantamos el dataset con target ya calculado

In [8]:
df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
df.shape

(31362, 19)

##### Preprocesamiento a la minima expresión :)

In [9]:
##### aplicamos OHE
df = preprocesamiento.aplicarOHE(df)
df.shape

(31362, 162)

### Feature Engineering

##### Neural Prophet

In [10]:
neural_prophet_fe = pd.read_csv("./datasets/features_neuralprophet_completo.csv", sep=',', encoding='utf-8')
neural_prophet_fe['ds'] = pd.to_datetime(neural_prophet_fe['ds'], errors='coerce')
# Versión alternativa más robusta:
neural_prophet_fe['periodo'] = neural_prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
neural_prophet_fe = neural_prophet_fe[['periodo', 'product_id', 'trend', "season_yearly", "season_monthly"]]
df = df.merge(neural_prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 165)

##### Prophet

In [11]:
prophet_fe = pd.read_csv("./datasets/prophet_features_tn_zscore.csv", sep=',', encoding='utf-8')
prophet_fe['ds'] = pd.to_datetime(prophet_fe['ds'], errors='coerce')
prophet_fe['periodo'] = prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
prophet_fe = prophet_fe[['periodo', 'product_id', 'trend_add', "yearly_add", "additive_terms", 'trend_mult', 'yearly_mult', 'multiplicative_terms']]
df = df.merge(prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 171)

##### FE Moviles

In [12]:
df = feature_engineering.get_lags(df, "tn", 201912)
df = feature_engineering.get_delta_lags(df, "tn", 24)
df = feature_engineering.get_rolling_means(df, "tn", 201912)
df = feature_engineering.get_rolling_stds(df, "tn", 201912)
df = feature_engineering.get_rolling_mins(df, "tn", 201912)
df = feature_engineering.get_rolling_maxs(df, "tn", 201912)
df = feature_engineering.get_rolling_medians(df, "tn", 201912)
df = feature_engineering.get_rolling_skewness(df, "tn", 201912)
df = feature_engineering.get_autocorrelaciones(df, "tn", 201912)
df = feature_engineering.add_delta_ma_ratios(df, "tn", 24)
df.shape

(31362, 805)

##### FE Moviles sobre otras variables

In [13]:
# #  stock final
# df = feature_engineering.get_lagsEspecificos(df, col='stock_final_zscore')
# df = feature_engineering.get_delta_lags_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_means_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_stds_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_medians_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_mins_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='stock_final_zscore')

# #  cust_request_qty
# df = feature_engineering.get_lagsEspecificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_delta_lags_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_means_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_stds_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_medians_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_mins_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='cust_request_qty_zscore')

##### FE Calendario

In [14]:
df = feature_engineering.generar_ids(df)
df = feature_engineering.get_componentesTemporales(df)
df = feature_engineering.get_anomaliasPoliticas(df)
# df = feature_engineering.descomposicion_serie_temporal(df, col='tn')
df.shape

(31362, 830)

##### FE sobre FE

In [15]:
df = feature_engineering.chatGPT_features_serie(df, "tn")
df = feature_engineering.mes_con_feriado(df)
df.shape

(31362, 859)

##### Variables Exogenas

In [16]:
df = feature_engineering.get_dolar(df)
df = feature_engineering.get_IPC(df)
df['ipc'] = df['ipc'].str.replace(',', '.').astype(float)
df['dolar'] = df['dolar'].str.replace(',', '.').astype(float)
# df.drop(columns=['ds'], inplace=True)
df.fillna(0, inplace=True) ##### EXPERIMENTAR
df = feature_engineering.correlacion_exogenas(df)
df = feature_engineering.get_mes_receso_escolar(df)
df.shape

(31362, 862)

##### EXP 2

-   Es el EXP 1 pero con linear_tree=True

Conclusion: dio horrible.


In [17]:
study, models, results = model_lgb.main_entrenar(df, VERSION="v12")

[I 2025-06-26 14:02:29,400] A new study created in memory with name: no-name-90c48cbe-492d-43e9-90b5-3df7e117b152


Features: ['product_id', 'periodo', 'nacimiento_producto', 'muerte_producto', 'mes_n', 'total_meses', 'producto_nuevo', 'ciclo_de_vida_inicial', 'sku_size', 'stock_final', 'tn', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'cat1_FOODS', 'cat1_HC', 'cat1_PC', 'cat1_REF', 'cat1_nan', 'cat2_ADEREZOS', 'cat2_CABELLO', 'cat2_DENTAL', 'cat2_DEOS', 'cat2_HOGAR', 'cat2_OTROS', 'cat2_PIEL1', 'cat2_PIEL2', 'cat2_PROFESIONAL', 'cat2_ROPA ACONDICIONADOR', 'cat2_ROPA LAVADO', 'cat2_ROPA MANCHAS', 'cat2_SOPAS Y CALDOS', 'cat2_TE', 'cat2_VAJILLA', 'cat2_nan', 'cat3_ACONDICIONADOR', 'cat3_Acond Bebe', 'cat3_Acond Mujer', 'cat3_Aero', 'cat3_Aji Picante', 'cat3_Almidon', 'cat3_Antigrasa', 'cat3_Azul', 'cat3_BARRAS', 'cat3_Baking Bags', 'cat3_Barbacoa', 'cat3_Barra', 'cat3_Baño', 'cat3_Bizcochuelo', 'cat3_CREMA', 'cat3_CUIDADO ESPECIAL', 'cat3_Caldo Cubo', 'cat3_Caldos Granulados', 'cat3_Cara', 'cat3_Cepilllo', 'cat3_Chimichurri', 'cat3_Concentrado', 'cat3_Cremoso', 'cat3_Cristalino', 

[I 2025-06-26 14:05:03,052] Trial 0 finished with value: 15.182921813785393 and parameters: {'max_depth': 9, 'learning_rate': 0.2536999076681772, 'n_estimators': 759, 'min_child_weight': 6, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'reg_alpha': 0.05808361216819946, 'reg_lambda': 0.8661761457749352}. Best is trial 0 with value: 15.182921813785393.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[27]	valid_0's l1: 21.5676
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[28]	valid_0's l1: 25.6977
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[30]	valid_0's l1: 21.763
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[50]	valid_0's l1: 14.786
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[57]	valid_0's l1: 14.6243
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[45]	valid_0's l1: 15.0878
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[117]	valid_0's l1: 12.2931
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[118]	valid_0's l1

[I 2025-06-26 14:06:21,974] Trial 1 finished with value: 14.452486459406611 and parameters: {'max_depth': 13, 'learning_rate': 0.11114989443094977, 'n_estimators': 118, 'min_child_weight': 10, 'subsample': 0.9162213204002109, 'colsample_bytree': 0.6061695553391381, 'reg_alpha': 0.18182496720710062, 'reg_lambda': 0.18340450985343382}. Best is trial 1 with value: 14.452486459406611.


...

Did not meet early stopping. Best iteration is:
[489]	valid_0's l1: 3.86946
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[27]	valid_0's l1: 23.1619
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[36]	valid_0's l1: 21.0472
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[30]	valid_0's l1: 19.8725
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[41]	valid_0's l1: 14.7193
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[40]	valid_0's l1: 15.5975
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[50]	valid_0's l1: 14.8027
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[150]	valid_0's l1: 11.4526
Training until validation scores don't improve for 30 rounds
E

[I 2025-06-26 14:11:09,437] Trial 3 finished with value: 13.75333465413419 and parameters: {'max_depth': 11, 'learning_rate': 0.14447746112718687, 'n_estimators': 279, 'min_child_weight': 6, 'subsample': 0.7962072844310213, 'colsample_bytree': 0.5232252063599989, 'reg_alpha': 0.6075448519014384, 'reg_lambda': 0.17052412368729153}. Best is trial 2 with value: 12.786034158485974.


Did not meet early stopping. Best iteration is:
[279]	valid_0's l1: 3.80729
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[20]	valid_0's l1: 24.1837
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[13]	valid_0's l1: 23.1166
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[23]	valid_0's l1: 26.9737
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[27]	valid_0's l1: 15.1422
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[34]	valid_0's l1: 16.5528
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[36]	valid_0's l1: 17.1138
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[38]	valid_0's l1: 13.6498
Training until validation scores don't improve for 30 rounds
Ea

[I 2025-06-26 14:13:11,848] Trial 4 finished with value: 15.976239601151368 and parameters: {'max_depth': 4, 'learning_rate': 0.2521267904777921, 'n_estimators': 970, 'min_child_weight': 9, 'subsample': 0.6523068845866853, 'colsample_bytree': 0.5488360570031919, 'reg_alpha': 0.6842330265121569, 'reg_lambda': 0.4401524937396013}. Best is trial 2 with value: 12.786034158485974.



🏆 Mejores parámetros:
max_depth: 8
learning_rate: 0.05958389350068958
n_estimators: 489
min_child_weight: 3
subsample: 0.8059264473611898
colsample_bytree: 0.569746930326021
reg_alpha: 0.29214464853521815
reg_lambda: 0.3663618432936917
boosting_type: gbdt
objective: regression
metric: mae
verbosity: -1
deterministic: True
force_col_wise: True

Entrenando modelo final...
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[489]	valid_0's l1: 3.67743
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[489]	valid_0's l1: 3.77169
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[489]	valid_0's l1: 3.70015
Ensemble MAE Val: 3.6225

🎯 MAE Test Ensemble: 3.7440

Top 5 mejores trials:
1. MAE: 12.7860
2. MAE: 13.7533
3. MAE: 14.4525
4. MAE: 15.1829
5. MAE: 15.9762


In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Supongamos que querés predecir para los períodos 202011 y 202012
periodos_future = [201911, 201912]
df_future = df[df['periodo'].isin(periodos_future)].copy()

# Asegurate de tener las mismas columnas que en entrenamiento
features = [col for col in df.columns if col not in ['target']]

# Predecir
future_preds = model_lgb.predict_with_ensemble(models, df_future, features)

# Si querés evaluar el MAE (solo si tenés target en esos períodos)
if 'target' in df_future:
    mae_future = mean_absolute_error(df_future['target'], future_preds)
    print(f"📊 MAE Future: {mae_future:.4f}")

# Guardar las predicciones
df_future['pred_ensemble'] = future_preds

📊 MAE Future: 28.8536


In [19]:
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")
df_future = df_future[df_future['periodo'] == 201912]
df_future = df_future[df_future['product_id'].isin(productos_ok['product_id'].unique())]
df_future = df_future[['product_id', 'pred_ensemble']]
df_future


,product_id,pred_ensemble
30476,20001,1553.396549
30477,20002,1108.824228
30478,20003,635.566857
30479,20004,535.935107
30480,20005,639.599844
...,...,...
31355,21263,-4.356842
31357,21265,1.240013
31358,21266,-1.558357
31359,21267,-3.819683


In [ ]:
promedios = model_lgb.promedio_12_meses_780p()
df_future = df_future.merge(promedios, on='product_id', how='left')
df_future.rename(columns={'tn_x': 'tn', 'tn_y': 'tn_12m'}, inplace=True)

In [ ]:
df_future.rename(columns={'pred_ensemble': 'tn', 'tn': 'tn_12m', 'tn_12m': 'mean'}, inplace=True)


,product_id,tn,tn_12m,mean
0,20001,1553.396549,1454.732720,1454.732720
1,20002,1108.824228,1175.437142,1175.437142
2,20003,635.566857,784.976407,784.976407
3,20004,535.935107,627.215328,627.215328
4,20005,639.599844,668.270104,668.270104
...,...,...,...,...
775,21263,-4.356842,0.029993,0.029993
776,21265,1.240013,0.089541,0.089541
777,21266,-1.558357,0.094659,0.094659
778,21267,-3.819683,0.092835,0.092835


In [40]:
df_future.drop(columns=['mean'], inplace=True)
df_future['tn'] = df_future['tn'].mask(df_future['tn'] < 0, df_future['tn_12m'])
df_future.drop(columns=['tn_12m'], inplace=True)

In [41]:
df_future

,product_id,tn
0,20001,1553.396549
1,20002,1108.824228
2,20003,635.566857
3,20004,535.935107
4,20005,639.599844
...,...,...
775,21263,0.029993
776,21265,1.240013
777,21266,0.094659
778,21267,0.092835


guardo el archivo

In [42]:
df_future.to_csv("./outputs/predicciones_exp_03_lgb_fe_v1.csv", index=False, sep=',')

Feature Importance

In [33]:
df_aux = df.drop(columns=['target'])


models_aux = models.copy()
count = 1

for model in models_aux:
    
    
    feature_importances = pd.DataFrame({
        'feature': df_aux.columns,
        'importance':  model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    display(feature_importances)
    
    feature_importances.to_csv(f"./feature_importance/exp03_{count}.csv", index=False, sep=',')
    count += 1
    

,feature,importance
165,yearly_add,220
164,trend_add,183
849,tn_seasonal_naive,173
167,trend_mult,144
837,tn_seasonality,141
...,...,...
39,cat3_Aji Picante,0
20,cat2_CABELLO,0
6,producto_nuevo,0
18,cat1_nan,0


,feature,importance
165,yearly_add,193
164,trend_add,177
849,tn_seasonal_naive,163
12,cust_request_qty,143
167,trend_mult,138
...,...,...
47,cat3_Baño,0
325,tn_delta_lag6_lag22,0
17,cat1_REF,0
32,cat2_TE,0


,feature,importance
165,yearly_add,222
164,trend_add,182
849,tn_seasonal_naive,172
837,tn_seasonality,139
12,cust_request_qty,130
...,...,...
202,tn_lag_33,0
811,year_end,0
57,cat3_Cremoso,0
18,cat1_nan,0
